# Your First Click Model: Click Thru Rate

This section examines the session data and computes the probability of relevance using Click-Thru-Rate. Roughly the number of clicks divided by the number of sessions. Then we examine wheter there's position bias in that data - that is, consider perhaps that some documents have a higher CTR only because they show up higher in the search results.

In [1]:
! cd ../data/retrotech && head signals.csv

import random
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
from aips import *
from session_gen import SessionGenerator
import os
from IPython.core.display import display,HTML

import matplotlib.pyplot as plt
import numpy as np
# if using a Jupyter notebook, includue:
%matplotlib inline

"query_id","user","type","target","signal_time"
"u2_0_1","u2","query","nook","2019-07-31 08:49:07.3116"
"u2_1_2","u2","query","rca","2020-05-04 08:28:21.1848"
"u3_0_1","u3","query","macbook","2019-12-22 00:07:07.0152"
"u4_0_1","u4","query","Tv antenna","2019-08-22 23:45:54.1030"
"u5_0_1","u5","query","AC power cord","2019-10-20 08:27:00.1600"
"u6_0_1","u6","query","Watch The Throne","2019-09-18 11:59:53.7470"
"u7_0_1","u7","query","Camcorder","2020-02-25 13:02:29.3089"
"u9_0_1","u9","query","wireless headphones","2020-04-26 04:26:09.7198"
"u10_0_1","u10","query","Xbox","2019-09-13 16:26:12.0132"


In [2]:
def all_sessions():
    import glob
    sessions = pd.concat([pd.read_csv(f, compression='gzip')
                          for f in glob.glob('*_sessions.gz')])
    return sessions.rename(columns={'clicked_doc_id': 'doc_id'})
    
sessions = all_sessions()
sessions

,sess_id,query,rank,doc_id,clicked
0,2,dryer,0.0,12505451713,False
1,2,dryer,1.0,84691226727,True
2,2,dryer,2.0,883049066905,False
3,2,dryer,3.0,14381196320,False
4,2,dryer,4.0,74108056764,False
...,...,...,...,...,...
149995,55001,blue ray,25.0,22265004517,False
149996,55001,blue ray,26.0,885170038875,False
149997,55001,blue ray,27.0,786936817232,False
149998,55001,blue ray,28.0,600603132872,False


In [3]:
products = fetch_products(doc_ids=sessions['doc_id'].unique())

# Listing 11.03

Viewing session 2 of query `transformers dark of the moon` in retrotech. Here we inspect one of the sessions. We encourage you to examine other sessions

In [4]:
QUERY='transformers dark of the moon'
query_sessions = sessions[sessions['query'] == QUERY]
query_sessions[query_sessions['sess_id'] == 2]

,sess_id,query,rank,doc_id,clicked
0,2,transformers dark of the moon,0.0,47875842328,False
1,2,transformers dark of the moon,1.0,24543701538,True
2,2,transformers dark of the moon,2.0,25192107191,False
3,2,transformers dark of the moon,3.0,47875841420,False
4,2,transformers dark of the moon,4.0,786936817218,False
5,2,transformers dark of the moon,5.0,47875842335,False
6,2,transformers dark of the moon,6.0,47875841406,False
7,2,transformers dark of the moon,7.0,97360810042,False
8,2,transformers dark of the moon,8.0,24543750949,False
9,2,transformers dark of the moon,9.0,36725235564,False


# Listing 11.04

Simple CTR based judgments for our query. We compute the CTR by taking the number of clicks for a document relative to the number of unique sessions the doc appears in for that query.

In [5]:
QUERY='transformers dark of the moon'
query_sessions = sessions[sessions['query'] == QUERY]

click_counts = query_sessions.groupby('doc_id')['clicked'].sum()
sess_counts = query_sessions.groupby('doc_id')['sess_id'].nunique()
ctrs = click_counts / sess_counts

ctrs.sort_values(ascending=False)

doc_id
97360810042     0.0824
47875842328     0.0734
47875841420     0.0434
24543701538     0.0364
25192107191     0.0352
786936817218    0.0236
97363560449     0.0192
47875841406     0.0160
400192926087    0.0124
47875842335     0.0106
97363532149     0.0084
93624956037     0.0082
36725235564     0.0082
47875841369     0.0074
24543750949     0.0062
dtype: float64

# Figure 11.2

Source code to render CTR judgment's ideal relevance ranking for `transformers dark of the moon`. In other words, our search results ordered from highest CTR to lowest.

In [6]:
render_judged(products,
              ctrs.to_frame(name='ctr').sort_values('ctr', ascending=False),
              grade_col='ctr',
              label=f"Click-Thru-Rate Judgments for q={QUERY}")

,ctr,image,upc,name,shortDescription
0,0.0824,,97360810042,Transformers: Dark of the Moon - Blu-ray Disc,\N
1,0.0734,,47875842328,Transformers: Dark of the Moon Stealth Force Edition - Nintendo Wii,Transform into an epic hero or a vehicular villain
2,0.0434,,47875841420,Transformers: Dark of the Moon Decepticons - Nintendo DS,Transform into an epic hero or a vehicular villain
3,0.0364,,24543701538,The A-Team - Widescreen Dubbed Subtitle AC3 - Blu-ray Disc,\N
4,0.0352,,25192107191,Fast Five - Widescreen - Blu-ray Disc,\N
5,0.0236,,786936817218,Pirates Of The Caribbean: On Stranger Tides (3-D) - Blu-ray 3D,\N
6,0.0192,,97363560449,Transformers: Dark of the Moon - Widescreen Dubbed Subtitle - DVD,\N
7,0.0160,,47875841406,Transformers: Dark of the Moon Autobots - Nintendo DS,Transform into an epic hero or a vehicular villain
8,0.0124,,400192926087,Transformers: Dark of the Moon - Original Soundtrack - CD,\N
9,0.0106,,47875842335,Transformers: Dark of the Moon Stealth Force Edition - Nintendo 3DS,Transform into an epic hero or a vehicular villain


# Figure 11.3

Source code to render CTR ideal relevance ranking for `dryer`. Ordering the highest CTR result to the lowest.

In [7]:
QUERY='dryer'
query_sessions = sessions[sessions['query'] == QUERY]

click_counts = query_sessions.groupby('doc_id')['clicked'].sum()
doc_counts = query_sessions.groupby('doc_id')['sess_id'].nunique()
ctrs = click_counts / doc_counts

render_judged(products,
              ctrs.to_frame(name='ctr').sort_values('ctr', ascending=False),
              grade_col='ctr',
              label=f"Click-Thru-Rate Judgments for q={QUERY}")

,ctr,image,upc,name,shortDescription
0,0.1608,,84691226727,GE - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White,Rotary electromechanical controls; 3 cycles; 3 heat selections; DuraDrum interior; Quiet-By-Design
1,0.0816,,84691226703,Hotpoint - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White-on-White,Rotary controls; 3 cycles; 3 temperature settings; DuraDrum interior
2,0.0710,,12505451713,Frigidaire - Semi-Rigid Dryer Vent Kit - Silver,Expandable vent; custom fitted ends and clamps
3,0.0576,,783722274422,The Independent - Widescreen Subtitle - DVD,\N
4,0.0572,,883049066905,Whirlpool - Affresh Washer Cleaner,Package include 3 tablets; removes and prevents odor-causing residue; compatible with all high-efficiency washers
5,0.0552,,77283045400,Hello Kitty - Hair Dryer - Pink,1875 watts of power; high and low heat settings; cool shot button; detachable styling nozzle
6,0.0546,,74108056764,Conair - Infiniti Ionic Cord-Keeper Hair Dryer - Light Purple,1875 watts; dual voltage; 2 heat and speed settings
7,0.0540,,665331101927,Everything in Static - CD,\N
8,0.0536,,12505525766,Smart Choice - 6' 30 Amp 3-Prong Dryer Cord,Heavy-duty PVC insulation; strain relief safety clamp
9,0.0470,,74108096487,Conair - Infiniti Cord-Keeper Professional Tourmaline Ionic Hair Dryer - Fuchsia,Tourmaline ceramic technology; ionic technology; 1875 watts; Cool Shot function; 3 heat settings; 2 speed settings; 5' retractable cord; includes diffuser


## Listing 11.5

Computing the global CTR of each rank per search ranking to consider whether the click data is biased by position. We look over every search to see what the CTR is when a document is placed in a specific rank.

In [8]:
num_sessions = len(sessions['sess_id'].unique())
global_ctrs = sessions.groupby('rank')['clicked'].sum() / num_sessions
global_ctrs

rank
0.0     0.249727
1.0     0.142673
2.0     0.084218
3.0     0.063073
4.0     0.056255
5.0     0.042255
6.0     0.033236
7.0     0.038000
8.0     0.020964
9.0     0.017364
10.0    0.013982
11.0    0.018582
12.0    0.015982
13.0    0.014509
14.0    0.012327
15.0    0.010200
16.0    0.011782
17.0    0.007891
18.0    0.007273
19.0    0.008145
20.0    0.006236
21.0    0.004473
22.0    0.005455
23.0    0.004982
24.0    0.005309
25.0    0.004364
26.0    0.005055
27.0    0.004691
28.0    0.005000
29.0    0.005400
Name: clicked, dtype: float64

## Listing 11.6

We look at the documents for our query, and notice that certain ones tend to appear higher and others tend to appear lower. If irrelevant ones dominate the top listings, position bias will dominate our training data

In [13]:
QUERY='transformers dark of the moon'
query_sessions = sessions[sessions['query'] == QUERY]

avg_rank = query_sessions.groupby('doc_id')['rank'].mean()

avg_rank.sort_values(ascending=True)

doc_id
47875842328      0.9808
24543701538      1.8626
25192107191      2.6596
47875841420      3.5344
786936817218     4.4444
47875842335      5.2776
47875841406      6.1378
97360810042      7.0130
24543750949      7.8626
36725235564      8.6854
47875841369      9.5796
97363560449     10.4304
93624956037     11.3298
97363532149     12.1494
400192926087    13.0526
Name: rank, dtype: float64

In [11]:
render_judged(products, 
              avg_rank.reset_index().sort_values('rank'),
              grade_col='rank',
              label=f"Typical Search Session for q={QUERY}")

,rank,image,upc,name,shortDescription
0,0.9808,,47875842328,Transformers: Dark of the Moon Stealth Force Edition - Nintendo Wii,Transform into an epic hero or a vehicular villain
1,1.8626,,24543701538,The A-Team - Widescreen Dubbed Subtitle AC3 - Blu-ray Disc,\N
2,2.6596,,25192107191,Fast Five - Widescreen - Blu-ray Disc,\N
3,3.5344,,47875841420,Transformers: Dark of the Moon Decepticons - Nintendo DS,Transform into an epic hero or a vehicular villain
4,4.4444,,786936817218,Pirates Of The Caribbean: On Stranger Tides (3-D) - Blu-ray 3D,\N
5,5.2776,,47875842335,Transformers: Dark of the Moon Stealth Force Edition - Nintendo 3DS,Transform into an epic hero or a vehicular villain
6,6.1378,,47875841406,Transformers: Dark of the Moon Autobots - Nintendo DS,Transform into an epic hero or a vehicular villain
7,7.0130,,97360810042,Transformers: Dark of the Moon - Blu-ray Disc,\N
8,7.8626,,24543750949,X-Men: First Class - Widescreen Dubbed Subtitle AC3 - Blu-ray Disc,\N
9,8.6854,,36725235564,"Samsung - 40"" Class - LCD - 1080p - 120Hz - HDTV",\N


In [12]:
QUERY='dryer'
query_sessions = sessions[sessions['query'] == QUERY]

avg_rank = query_sessions.groupby('doc_id')['rank'].mean()

avg_rank
render_judged(products, 
              avg_rank.reset_index().sort_values('rank'),
              grade_col='rank',
              label=f"Typical Search Session for q={QUERY}")

,rank,image,upc,name,shortDescription
0,1.9124,,12505451713,Frigidaire - Semi-Rigid Dryer Vent Kit - Silver,Expandable vent; custom fitted ends and clamps
1,2.8290,,84691226727,GE - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White,Rotary electromechanical controls; 3 cycles; 3 heat selections; DuraDrum interior; Quiet-By-Design
2,3.5726,,883049066905,Whirlpool - Affresh Washer Cleaner,Package include 3 tablets; removes and prevents odor-causing residue; compatible with all high-efficiency washers
3,4.4552,,84691226703,Hotpoint - 6.0 Cu. Ft. 3-Cycle Electric Dryer - White-on-White,Rotary controls; 3 cycles; 3 temperature settings; DuraDrum interior
4,5.1276,,74108056764,Conair - Infiniti Ionic Cord-Keeper Hair Dryer - Light Purple,1875 watts; dual voltage; 2 heat and speed settings
5,5.9318,,77283045400,Hello Kitty - Hair Dryer - Pink,1875 watts of power; high and low heat settings; cool shot button; detachable styling nozzle
6,6.7610,,783722274422,The Independent - Widescreen Subtitle - DVD,\N
7,7.5138,,665331101927,Everything in Static - CD,\N
8,8.3308,,14381196320,The Mind Snatchers - DVD,\N
9,9.1230,,74108096487,Conair - Infiniti Cord-Keeper Professional Tourmaline Ionic Hair Dryer - Fuchsia,Tourmaline ceramic technology; ionic technology; 1875 watts; Cool Shot function; 3 heat settings; 2 speed settings; 5' retractable cord; includes diffuser
